In [ ]:
#source activate /software/conda/envs/SE3nv
import sys,os
script_dir = os.path.dirname(os.path.realpath(__file__))
import torch 
import glob
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import subprocess
%matplotlib inline
pd.options.display.max_colwidth = 1000  

sys.path.append(script_dir+'/util')
import parsers, util, kinematics
import amino_acids as AA
import symmetry
from scipy.spatial.transform import Rotation as R


sys.path.append(script_dir+'LA_scripts/')
from get_rmsds_functions import TMalign
from get_rmsds_functions import get_RMSD
import check_interactions_to_lig


## centering pdb

In [ ]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [ ]:
pdbs = ['CHD_17971.pdb', 'CHD_4019.pdb', 'CHD_8651.pdb']
for pdb in pdbs:
    pdb = f'{home_dir}/{pdb}'

    parsed = parsers.parse_pdb(pdb)

    xyz_native = parsed['xyz']
    seq_native = parsed['seq']

    # center it at the origin and save for reference 
    outdir = f'{home_dir}/asu_pdbs/'
    xyz_native_centered = xyz_native - xyz_native[:,1,:].mean(axis=0)

    seq_torch = torch.from_numpy(seq_native)

    pdb = os.path.basename(pdb)[:-4] #remove .pdb
    util.writepdb(os.path.join(outdir, f'{pdb}_centered.pdb'), 
                  torch.from_numpy(xyz_native_centered), 
                  torch.ones_like(seq_torch),
                  seq_torch)

    ### oritened the cavity parrallel to the z-axis
    S = symmetry.SymGen('C3',10,True) # recenter=10, but parameter not used for Cn
    L_monomer = 40
    rmsds = []

    xyz = torch.clone(torch.from_numpy(xyz_native_centered))
    seq = torch.clone(torch.from_numpy(seq_native))

    i1 = 0
    i2 = i1 + L_monomer
    i3 = i2 + L_monomer

    c1 = xyz[i1,1,:]
    c2 = xyz[i2,1,:]
    c3 = xyz[i3,1,:]


    v1 = c2 - c1
    v1 /= torch.norm(v1)

    v2 = c3 - c1
    v2 /= torch.norm(v2)

    v3 = torch.cross(v1,v2)
    v3 /= torch.norm(v3)

    z = np.array([0,0,1])
    Rz = torch.from_numpy( rotation_matrix_from_vectors(v3,z) )

    # rotate coordinates 
    shape = xyz.shape
    xyz_oriented = xyz.reshape(-1,3).float()@Rz.T.float()
    xyz_oriented = xyz_oriented.reshape(shape)

    fpath = os.path.join(outdir, f'{pdb}_centered_oriented.pdb')
    if not os.path.exists(fpath):
        util.writepdb(fpath, xyz_oriented, torch.ones_like(seq), seq)
    sections = {'p1':0,'p2':40,'p3':80}

    for indi_block in sections:

        # slice out the ASU
        start = sections[indi_block]
        xyz_asu = xyz_oriented[start:start+L_monomer]
        xyz_asu = torch.cat([xyz_asu]*3)
        seq_asu = seq[start:start+L_monomer]
        seq_asu = torch.cat([seq_asu]*3)

        # symmetrize the sliced ASU  
        xyz_symm,seq_symm = S.apply_symmetry(xyz_asu, seq_asu)

        # calculate RMSD 
        A = xyz_symm[:,:3,:].numpy().reshape(-1,3)
        B = xyz_oriented[:,:3,:].numpy().reshape(-1,3)
        rmsd,_,_ = kinematics.np_kabsch(A,B)

        chains = ['A']*40 + ['B']*40 + ['C']*40
        util.writepdb(os.path.join(outdir, f'{pdb}_c3_{indi_block}.pdb'), 
                      xyz_symm,
                      torch.ones_like(seq_symm),
                      seq_symm)

        rmsds.append((indi_block,rmsd))


### RFdiffusion scaffolding
// using RFdiffusion to generate scaffold

In [ ]:
python = '/path/to/rfdiffusion/python-envs'
inference = '/net/databases/diffusion/github_repo/rf_diffusion/run_inference.py'
cmd = f"{python} {inference} -python /net/databases/diffusion/github_repo/rf_diffusion/run_inference.py scaffoldguided.scaffoldguided=True scaffoldguided.scaffold_dir=./adj_ss_pt/ inference.output_prefix=./pdb_out/adj_pt_design inference.input_pdb=./pdb/1622.pdb contigmap.contigs=[\'A1-17,50,A22-57,49,A62-96,47,A102-120\'] contigmap.inpaint_seq=[\'A3,A6,A9,A10,A13-14,A16,A24,A27-28,A31-32,A34-35,A38-39,A43,A46,A49-50,A52-54,A56-57,A64,A68,A71-72,A74-76,A78-79,A83,A85-87,A89-90,A92-94,A96,A104,A107-108,A111-112,A114-115,A118-119\'] potentials.guiding_potentials=[\'type:binder_ncontacts\'] potentials.guide_scale=2 potentials.guide_decay='quadratic' denoiser.noise_scale_ca=0.5 denoiser.noise_scale_frame=0.5 inference.num_designs=200 inference.final_step=50"

### MPNN
// sequence design newly generated scaffold

In [ ]:
python = 'path/to/MPNN/python-envs/'
mpnn = f'{script_dir}/LA_scripts/toolkits/ligand_proteinmpnn/protein_mpnn_run.py' 


cmd =f'{python} {mpnn} --out_folder ./ --num_seq_per_target 8 --pdb_path pdb/P1D7_B9.pdb --pack_side_chains 1 --num_packs 1 --sampling_temp "0.1 0.2" --use_ligand 1 --tied_positions_jsonl pdb/tied_pos.jsonl --ligand_params_path pdb/CHD.params'

### AlphaFold each mononer
// af2 validate folding likelihood

In [ ]:
python = '/path/to/af2/python-envs'
run_af2 = f'{script_dir}/LA_scripts/toolkits/run_af2.py'
cmd = f"{python} {run_af2} -python {run_af2} -pdb {pdb}"